<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-xxxxxx'
PROJECT = 'qwiklabs-gcp-xxxxxx'
REGION = 'us-west1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [6]:
# Create SQL query using natality data after the year 2000
from google.cloud import bigquery
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

## Lab Task #1

Sample the BigQuery resultset (above) so that you have approximately 12,000 training examples and 3000 evaluation examples.
The training and evaluation datasets have to be well-distributed (not all the babies are born in Jan 2005, for example)
and should not overlap (no baby is part of both training and evaluation datasets).

Hint (highlight to see): <p style='color:white'>You will use MOD() on the hashmonth to divide the dataset into non-overlapping training and evaluation datasets, and RAND() to sample these to the desired size.</p>

In [8]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bigquery.Client().query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,8904940584331855459,344191
1,6691862025345277042,338820
2,2126480030009879160,344357
3,5934265245228309013,324598
4,1005902328334352553,337637


In [9]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) = 3 AND RAND() < 0.0005"
traindf = bigquery.Client().query(trainQuery).to_dataframe()
evaldf = bigquery.Client().query(evalQuery).to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13304 examples in the train dataset and 3319 in the eval dataset


## Lab Task #2

Use Pandas to:
* Clean up the data to remove rows that are missing any of the fields.
* Simulate the lack of ultrasound.
* Change the plurality column to be a string.

Hint (highlight to see): <p>
Filtering:
<pre style='color:white'>
df = df[df.weight_pounds > 0]
</pre>
Lack of ultrasound:
<pre style='color:white'>
nous = df.copy(deep=True)
nous['is_male'] = 'Unknown'
</pre>
Modify plurality to be a string:
<pre style='color:white'>
twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
df['plurality'].replace(twins_etc, inplace=True)
</pre>
</p>

In [15]:
# Create SQL query using natality data after the year 2000
from google.cloud import bigquery
trainQuery = """
SELECT * from (
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000 and is_male is not null and plurality is not null and weight_pounds is not null and gestation_weeks is not null)
WHERE MOD(hashmonth, 4) < 3 AND RAND() < 0.0005
"""

traindf = bigquery.Client().query(trainQuery).to_dataframe()
traindf.head()
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13151.000000,13151.000000,13151.000000,13151.000000,1.315100e+04
mean,7.231201,27.421869,1.035206,38.585811,4.398036e+18
std,1.337686,6.191823,0.192383,2.612589,2.786913e+18
min,0.568793,12.000000,1.000000,19.000000,1.244589e+17
25%,6.580799,22.000000,1.000000,38.000000,1.622638e+18
50%,7.312733,27.000000,1.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,1.000000,40.000000,7.108882e+18
max,12.687603,50.000000,4.000000,47.000000,9.183606e+18


In [26]:
trainQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) = 3 AND RAND() < 0.0005"
traindf = bigquery.Client().query(trainQuery).to_dataframe()
evaldf = bigquery.Client().query(evalQuery).to_dataframe()

In [27]:
evaldf.head()
evaldf.describe()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
count,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
    # clean up data we don't want to train on
    # in other words, users will have to tell us the mother's age
    # otherwise, our ML service won't work.
    # these were chosen because they are such good predictors
    # and because these are easy enough to collect
    df = df[df.weight_pounds > 0]
    df = df[df.mother_age > 0]
    df = df[df.gestation_weeks > 0]
    df = df[df.plurality > 0]
  
    # modify plurality field to be a string
    twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
    df['plurality'].replace(twins_etc, inplace=True)
  
    # now create extra rows to simulate lack of ultrasound
    nous = df.copy(deep=True)
    nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
    nous['is_male'] = 'Unknown'
    
    
    return pd.concat([df, nous])

In [29]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
traindf.head()


,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.686620,False,26,Single(1),39.0,6086613178753042412
1,10.688010,True,34,Single(1),39.0,3545707052733304728
3,7.131954,False,23,Single(1),40.0,1195438672706281328
4,6.102395,False,21,Twins(2),38.0,260598435387740869
5,6.624891,True,17,Single(1),40.0,524531196325542205


## Lab Task #3

Write the cleaned out data into CSV files.  Change the name of the Pandas dataframes (traindf, evaldf) appropriately.



In [30]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [31]:
%%bash
wc -l *.csv
head *.csv
tail *.csv

  0 eval.csv
 14 train.csv
 14 total
==> eval.csv <==

==> train.csv <==
6.686620406459999,False,26,Single(1),39.0,6086613178753042412
10.68801046176,True,34,Single(1),39.0,3545707052733304728
7.1319541757,False,23,Single(1),40.0,1195438672706281328
6.10239541216,False,21,Twins(2),38.0,260598435387740869
6.6248909731,True,17,Single(1),40.0,524531196325542205
8.59361897276,True,31,Single(1),39.0,700070369880083266
7.7492485093,True,25,Single(1),38.0,2700219941840496344
6.686620406459999,Unknown,26,Single(1),39.0,6086613178753042412
10.68801046176,Unknown,34,Single(1),39.0,3545707052733304728
7.1319541757,Unknown,23,Single(1),40.0,1195438672706281328
==> eval.csv <==

==> train.csv <==
6.6248909731,True,17,Single(1),40.0,524531196325542205
8.59361897276,True,31,Single(1),39.0,700070369880083266
7.7492485093,True,25,Single(1),38.0,2700219941840496344
6.686620406459999,Unknown,26,Single(1),39.0,6086613178753042412
10.68801046176,Unknown,34,Single(1),39.0,3545707052733304728
7.1319541757,Un

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License